In [0]:
%run ../../connections/connection_notebook

In [0]:
import os
import pyspark.pandas as pd
import pyspark.sql.functions as F
import pyspark.sql.types as T
import datetime
import json
print(datetime.datetime.today().strftime('%Y-%m-%d'))

2022-12-30


In [0]:
def listDir(dir_):
    return dbutils.fs.ls(dir_)

In [0]:
#dbutils.fs.ls(f'{transient_storage_mount_path}/corporate/{area_negocio}/processed/')
dataCarga = '20221229'#datetime.date.today().strftime('%Y%m%d')
area_negocio = "suprimentos/comprascentralizadas"
system_database = "corporate"

path_raw = f'{raw_storage_mount_path}/corporate/{area_negocio}'
path_struct = f'{structured_storage_mount_path}/corporate/{area_negocio}'

In [0]:
proccess = [
    'ME2L',
    'ME3L',
    'SE16N'
]
raw = { x: f'{path_raw}/{x}/{dataCarga}' for x in proccess}
struct = { x: f'{path_struct}/v0/{x}' for x in proccess}

#chave_se116n_strc = ['nr_rqsco_cmpra','cd_itm_rqsco']


chave_me2l = ['documento_de_compras','item']
chave_me3l = ['documento_de_compras','item']
chave_se116n = ['reqc','item_requisicao']
keys = [
    chave_me2l,
    chave_me3l,
    chave_se116n
]

In [0]:
dct_keys = {
    x:keys[y] for y,x in enumerate(proccess)
}

In [0]:
print(json.dumps(raw,  indent=4))
print(json.dumps(struct,  indent=4))

{
    "ME2L": "abfss://raw@[REDACTED].dfs.core.windows.net/corporate/suprimentos/comprascentralizadas/ME2L/20221229",
    "ME3L": "abfss://raw@[REDACTED].dfs.core.windows.net/corporate/suprimentos/comprascentralizadas/ME3L/20221229",
    "SE16N": "abfss://raw@[REDACTED].dfs.core.windows.net/corporate/suprimentos/comprascentralizadas/SE16N/20221229"
}
{
    "ME2L": "abfss://structured@[REDACTED].dfs.core.windows.net/corporate/suprimentos/comprascentralizadas/v0/ME2L",
    "ME3L": "abfss://structured@[REDACTED].dfs.core.windows.net/corporate/suprimentos/comprascentralizadas/v0/ME3L",
    "SE16N": "abfss://structured@[REDACTED].dfs.core.windows.net/corporate/suprimentos/comprascentralizadas/v0/SE16N"
}


## Funções de Apoio

In [0]:
def schemaList(df):
    listCampo = []
    for i  in list(df.schema):
        listCampo.append(str(i).replace('StructField(','').replace("'",'').replace('Type()','').split(',')[:2])
    return listCampo
def changeDataTypeFromSchema(df1,df2):
    schema1 = schemaList(df1)
    schema2 = schemaList(df2)
    
    listaConvert = []
    for pos in range(len(schema1)):
        #print(f'{schema1[pos]} -  {schema2[pos]}' )
        #print(schema1[pos][1].lower() == schema2[pos][1].lower())
        if(not(schema1[pos][1].lower() == schema2[pos][1].lower())):
            listaConvert.append(schema1[pos])
    return listaConvert    

# Cria um data frame com os campos renomeados removendo espaços

In [0]:
def createAndRenameColumns(proc):
    try:
        df  = ( 
            spark
            .read
            .parquet(raw[proc])    
        )#.groupBy('Pedido    ').count().where('count>1').display()
        
        for column in df.columns:
            df  = df.withColumnRenamed(column,column.replace(' ','').lower())
        columns_df = df.columns
        try:
            dbutils.fs.ls(struct[proc])
        except:
            print('1º Vez rodando, criando tabela delta baseada no processo '+proc)
            dfst = spark.createDataFrame([],df.schema)
            dfst.write.format('delta').mode('overwrite').option("mergeSchema", "true").save(struct[proc])
        return df
    except Exception as ex:
        return str(ex)

# Função de processamento dos dados na camada Structured

In [0]:
def processarDataframeStructured(df,proc,key):
    try:
        print(proc)
        df_struct = spark.read.format('delta').load(struct[proc])
        listChange = changeDataTypeFromSchema(df_struct,df)

        for campo in listChange:
            df = df.withColumn(campo[0],F.col(campo[0]).cast(campo[1]))

        df_raw_key = df.select(F.concat(*key).alias('key_2'))
        df_struct = df_struct.withColumn('key',F.concat(*key))
        
                
        if(df_struct.first() == None):
            df.write.format('delta').mode('overwrite').option("mergeSchema", "true").save(struct[proc])
            #return True
        else:    
            print("Existem registros")
            df_struct_key = df_struct.join(df_raw_key, on=[df_raw_key.key_2 == df_struct.key] ,how='outer' )

            df_novos_key = df_struct_key.filter(F.col('key').isNull()).select(F.col('key_2').alias('key'))

            df_old = (
                df_struct
                .filter(~F.col('key').isin(df_novos_key.select(F.collect_list(F.col('key')).alias('key')).first().key)
               )
                .drop('key')
                .drop('etl_date')
            )
            print(df_novos_key.count())
            df = df.withColumn('key',F.concat(*key))
            df_novos = df.join(df_novos_key,on=['key'],how='inner').drop('key').drop('etl_date')
            df_final = df_old.union(df_novos).withColumn('etl_date',F.current_date().cast('string'))
            print(df.count())
            print(df_final.count())
            df_final.write.format('delta').mode('overwrite').option("mergeSchema", "true").save(struct[proc])
        #print(df_final.count())
        return True
    except Exception as ex:
        return str(ex)

# Chamada da função de processamento

In [0]:
processo_final = []
for x in proccess:
    try:
        dbutils.fs.ls(raw[x])
        processo_final.append(x)
    except:
        pass
processo_final

Out[34]: ['SE16N']

In [0]:
dct_keys[proc]

Out[43]: ['reqc', 'item_requisicao']

In [0]:
for proc in processo_final:
    print(proc)
    df_proc = createAndRenameColumns(proc)
    #print(df_proc)
    final = processarDataframeStructured(df_proc,proc,dct_keys[proc])
    print(final)

SE16N
SE16N
Existem registros
317292
317311
616675
True


In [0]:
df_struct = spark.read.format('delta').load(struct[proc])
listChange = changeDataTypeFromSchema(df_struct,df)

for campo in listChange:
    df = df.withColumn(campo[0],F.col(campo[0]).cast(campo[1]))

df_raw_key = df.select(F.concat(*key).alias('key_2'))
df_struct = df_struct.withColumn('key',F.concat(*key))


if(df_struct.first() == None):
    df.write.format('delta').mode('overwrite').option("mergeSchema", "true").save(struct[proc])
    #return True
else:    
    print("Existem registros")
    df_struct_key = df_struct.join(df_raw_key, on=[df_raw_key.key_2 == df_struct.key] ,how='outer' )

    df_novos_key = df_struct_key.filter(F.col('key').isNull()).select(F.col('key_2').alias('key'))

    df_old = (
        df_struct
        .filter(~F.col('key').isin(df_novos_key.select(F.collect_list(F.col('key')).alias('key')).first().key)
       )
        .drop('key')
        .drop('etl_date')
    )
    print(df_novos_key.count())
    df = df.withColumn('key',F.concat(*key))
    df_novos = df.join(df_novos_key,on=['key'],how='inner').drop('key').drop('etl_date')
    df_final = df_old.union(df_novos).withColumn('etl_date',F.current_date().cast('string'))
    print(df.count())
    print(df_final.count())
    df_final.write.format('delta').mode('overwrite').option("mergeSchema", "true").save(struct[proc])